In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import matplotlib.pyplot as plt
import torch
# Neural networks can be constructed using the torch.nn package.
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.sampler import SequentialSampler
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms

In [2]:
!ls 'drive/My Drive/Research/SAnet/session1_files'
%cd 'drive/My Drive/Research/SAnet/yolov5/yolov5'
!pwd

'session1 (100).jpg'  'session1 (207).jpg'  'session1 (313).jpg'
'session1 (101).jpg'  'session1 (208).jpg'  'session1 (314).jpg'
'session1 (102).jpg'  'session1 (209).jpg'  'session1 (315).jpg'
'session1 (103).jpg'  'session1 (20).jpg'   'session1 (316).jpg'
'session1 (104).jpg'  'session1 (210).jpg'  'session1 (317).jpg'
'session1 (105).jpg'  'session1 (211).jpg'  'session1 (318).jpg'
'session1 (106).jpg'  'session1 (212).jpg'  'session1 (319).jpg'
'session1 (107).jpg'  'session1 (213).jpg'  'session1 (31).jpg'
'session1 (108).jpg'  'session1 (214).jpg'  'session1 (320).jpg'
'session1 (109).jpg'  'session1 (215).jpg'  'session1 (321).jpg'
'session1 (10).jpg'   'session1 (216).jpg'  'session1 (322).jpg'
'session1 (110).jpg'  'session1 (217).jpg'  'session1 (323).jpg'
'session1 (111).jpg'  'session1 (218).jpg'  'session1 (324).jpg'
'session1 (112).jpg'  'session1 (219).jpg'  'session1 (325).jpg'
'session1 (113).jpg'  'session1 (21).jpg'   'session1 (326).jpg'
'session1 (114).jpg'  'ses

In [3]:
import argparse

import torch.backends.cudnn as cudnn
from google.colab.patches import cv2_imshow
from utils import google_utils
from utils.datasets import *
from utils.utils import *

import glob
import math
import os
import random
import shutil
import time
from pathlib import Path
from threading import Thread

import cv2
import numpy as np
import torch
from PIL import Image, ExifTags
from torch.utils.data import Dataset
from tqdm import tqdm

from utils.utils import xyxy2xywh, xywh2xyxy

help_url = 'https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data'
img_formats = ['.bmp', '.jpg', '.jpeg', '.png', '.tif', '.dng']
vid_formats = ['.mov', '.avi', '.mp4', '.mpg', '.mpeg', '.m4v', '.wmv', '.mkv']



class LoadImages1:  # for inference
    def __init__(self, path, session, img_size=416):
        path = str(Path(path))  # os-agnostic
        files = []
        if os.path.isdir(path):
            files = sorted(glob.glob(os.path.join(path, '*.*')))
        elif os.path.isfile(path):
            files = [path]

        # images = [x for x in files if os.path.splitext(x)[-1].lower() in img_formats]
        images = []
        if session == 'session1':          
            for i in range(353):
                images.append(os.path.join(path,'session1 ('+str(i+1)+').jpg') )

        elif session == 'session2':          
            for i in range(386):
                images.append(os.path.join(path,'session2 ('+str(i+1)+').jpg') )


        videos = [x for x in files if os.path.splitext(x)[-1].lower() in vid_formats]
        nI, nV = len(images), len(videos)

        self.img_size = img_size
        self.files = images + videos
        self.nF = nI + nV  # number of files
        self.video_flag = [False] * nI + [True] * nV
        self.mode = 'images'
        if any(videos):
            self.new_video(videos[0])  # new video
        else:
            self.cap = None
        assert self.nF > 0, 'No images or videos found in %s. Supported formats are:\nimages: %s\nvideos: %s' % \
                            (path, img_formats, vid_formats)

    def __iter__(self):
        self.count = 0
        return self

    def __next__(self):
        if self.count == self.nF:
            raise StopIteration
        path = self.files[self.count]

        if self.video_flag[self.count]:
            # Read video
            self.mode = 'video'
            ret_val, img0 = self.cap.read()
            if not ret_val:
                self.count += 1
                self.cap.release()
                if self.count == self.nF:  # last video
                    raise StopIteration
                else:
                    path = self.files[self.count]
                    self.new_video(path)
                    ret_val, img0 = self.cap.read()

            self.frame += 1
            print('video %g/%g (%g/%g) %s: ' % (self.count + 1, self.nF, self.frame, self.nframes, path), end='')

        else:
            # Read image
            self.count += 1
            img0 = cv2.imread(path)  # BGR
            assert img0 is not None, 'Image Not Found ' + path
            print('image %g/%g %s: ' % (self.count, self.nF, path), end='')

        # Padded resize
        img = letterbox(img0, new_shape=self.img_size)[0]

        # Convert
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)

        # cv2.imwrite(path + '.letterbox.jpg', 255 * img.transpose((1, 2, 0))[:, :, ::-1])  # save letterbox image
        return path, img, img0, self.cap

    def new_video(self, path):
        self.frame = 0
        self.cap = cv2.VideoCapture(path)
        self.nframes = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

    def __len__(self):
        return self.nF  # number of files





def detect(source_path,session,save_img=False):
    out, source, weights, view_img, save_txt, imgsz = \
        opt.output, source_path, opt.weights, opt.view_img, opt.save_txt, opt.img_size
    webcam = source == '0' or source.startswith('rtsp') or source.startswith('http') or source.endswith('.txt')

    # if session == 'session1':
    #     out = '/content/drive/My Drive/Research/SAnet/session1_files'
    # elif session == 'session2':
    #     out = '/content/drive/My Drive/Research/SAnet/session2_files'

    # Initialize
    device = torch_utils.select_device(opt.device)
    if os.path.exists(out):
        shutil.rmtree(out)  # delete output folder
    os.makedirs(out)  # make new output folder
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    google_utils.attempt_download(weights)
    model = torch.load(weights, map_location=device)['model'].float()  # load to FP32
    # torch.save(torch.load(weights, map_location=device), weights)  # update model if SourceChangeWarning
    # model.fuse()
    model.to(device).eval()
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = torch_utils.load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
        modelc.to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = True
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz)
    else:
        save_img = True
        dataset = LoadImages1(source, session, img_size=imgsz)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    bounding_boxes_all_images = []
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = torch_utils.time_synchronized()
        pred = model(img, augment=opt.augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
        t2 = torch_utils.time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0 = path[i], '%g: ' % i, im0s[i].copy()
            else:
                p, s, im0 = path, '', im0s

            save_path = str(Path(out) / Path(p).name)
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  #  normalization gain whwh
            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string

                # Write results
                minx = 0 
                miny = 0
                maxx = 0
                maxy = 0

                box_num = 0
                bounding_boxes = {}
                for *xyxy, conf, cls in det:
                    box_num+=1
                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                    #print("Box num:", box_num, " label: ",names[int(cls)], " xyxy: ", int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]), "xywh: ", float(xywh[0]), float(xywh[1]), float(xywh[2]), float(xywh[3]))
                    if (bounding_boxes.get(names[int(cls)], None) == None):
                        bounding_boxes[names[int(cls)]] = [[int(xyxy[0]), int(xyxy[1]), abs(int(xyxy[2])-int(xyxy[0])), abs(int(xyxy[3])-int(xyxy[1]))]]
                    else:
                        bounding_boxes[names[int(cls)]].append([int(xyxy[0]), int(xyxy[1]), abs(int(xyxy[2])-int(xyxy[0])), abs(int(xyxy[3])-int(xyxy[1]))])


                    tlx,tly, brx, bry = int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])
                    if tlx < minx: 
                        minx = tlx 
                    if tly < miny:
                        miny = tly
                    if bry > maxy:
                        maxy = bry
                    if brx > maxx:
                        maxx = brx #crop_img = img[y:y+h, x:x+w]


                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        with open(save_path[:save_path.rfind('.')] + '.txt', 'a') as file:
                            file.write(('%g ' * 5 + '\n') % (cls, *xywh))  # label format

                    if save_img or view_img:  # Add bbox to image
                        label = '%s %.2f' % (names[int(cls)], conf)
                        # plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)
            bounding_boxes_all_images.append(bounding_boxes)    
            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))
            im0 = im0[miny:maxy, minx:maxx]


            # Stream results
            if view_img:
                cv2_imshow( im0)
                not_showing = True
                #cv2.imshow(p, im0)
                
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'images':
                    # cv2.imwrite(save_path, im0)
                    not_saving = True
                else:
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*opt.fourcc), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        print('Results saved to %s' % os.getcwd() + os.sep + out)
        if platform == 'darwin':  # MacOS
            os.system('open ' + save_path)

    print('Done. (%.3fs)' % (time.time() - t0))
    return bounding_boxes_all_images

######################################################################################

class Options:
    def __init__(self):
      self.weights = 'weights/best.pt'
      self.source = 'inference/images'
      self.output = 'inference/output'
      self.img_size = 640
      self.conf_thres = 0.4
      self.iou_thres = 0.5
      self.fourcc = 'mp4v'
      self.device = ''
      self.view_img = False
      self.agnostic_nms = False
      self.augment = False      
      self.save_txt = False
      self.classes = None

opt = Options()
# # if __name__ == '__main__':
# parser = argparse.ArgumentParser()
# parser.add_argument('--weights', type=str, default='weights/yolov5s.pt', help='model.pt path')
# parser.add_argument('--source', type=str, default='inference/images', help='source')  # file/folder, 0 for webcam
# parser.add_argument('--output', type=str, default='inference/output', help='output folder')  # output folder
# parser.add_argument('--img-size', type=int, default=640, help='inference size (pixels)')
# parser.add_argument('--conf-thres', type=float, default=0.4, help='object confidence threshold')
# parser.add_argument('--iou-thres', type=float, default=0.5, help='IOU threshold for NMS')
# parser.add_argument('--fourcc', type=str, default='mp4v', help='output video codec (verify ffmpeg support)')
# parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
# parser.add_argument('--view-img', action='store_true', help='display results')
# parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
# parser.add_argument('--classes', nargs='+', type=int, help='filter by class')
# parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
# parser.add_argument('--augment', action='store_true', help='augmented inference')
# opt = parser.parse_args()
opt.img_size = check_img_size(opt.img_size)
print(opt)

opt.weights='weights/best.pt'
opt.view_img=False

# det_bboxes = None
# source_path = 'inference/images/session2 (210).jpg'
source_path = '../../jpg_files'
# with torch.no_grad():
#     det_bboxes = detect(source_path,'session1')

        # Update all models
        # for opt.weights in ['yolov5s.pt', 'yolov5m.pt', 'yolov5l.pt', 'yolov5x.pt', 'yolov3-spp.pt']:
        #    detect()
        #    create_pretrained(opt.weights, opt.weights)


In [4]:
# BASE_PATH = './'
ANNO_PATH = '../../annos/'
DATA_PATH = '../../jpg_files/'

with torch.no_grad():
    # det_boxes_session1
    source_path = DATA_PATH #os.path.join(DATA_PATH,'session1 ('+str(i+1)+').jpg')
    det_boxes_session1 = detect(source_path, 'session1')

    source_path = DATA_PATH #os.path.join(DATA_PATH,'session2 ('+str(i+1)+').jpg')
    det_boxes_session2 = detect(source_path, 'session2')

Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute o

image 1/353 ../../jpg_files/session1 (1).jpg: 

/content/drive/My Drive/Research/SAnet/yolov5/yolov5/utils/utils.py:580: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  i, j = (x[:, 5:] > conf_thres).nonzero().t()


384x640 3 blemisheds, 7 unblemisheds, 1 gloves, 1 belts, 1 bins, 1 heads, Done. (0.752s)
image 2/353 ../../jpg_files/session1 (2).jpg: 384x640 3 blemisheds, 7 unblemisheds, 1 gloves, 1 belts, 1 bins, 1 heads, Done. (0.036s)
image 3/353 ../../jpg_files/session1 (3).jpg: 384x640 4 blemisheds, 6 unblemisheds, 1 gloves, 1 belts, 1 bins, 1 heads, Done. (0.035s)
image 4/353 ../../jpg_files/session1 (4).jpg: 384x640 4 blemisheds, 6 unblemisheds, 1 gloves, 1 belts, 1 bins, 1 heads, Done. (0.036s)
image 5/353 ../../jpg_files/session1 (5).jpg: 384x640 4 blemisheds, 7 unblemisheds, 1 gloves, 1 belts, 1 bins, 1 heads, Done. (0.036s)
image 6/353 ../../jpg_files/session1 (6).jpg: 384x640 4 blemisheds, 7 unblemisheds, 1 gloves, 1 belts, 1 bins, 1 heads, Done. (0.035s)
image 7/353 ../../jpg_files/session1 (7).jpg: 384x640 4 blemisheds, 6 unblemisheds, 1 gloves, 1 belts, 1 bins, 1 heads, Done. (0.036s)
image 8/353 ../../jpg_files/session1 (8).jpg: 384x640 4 blemisheds, 6 unblemisheds, 1 gloves, 1 belts

In [ ]:
# det_boxes_session1[0]['glove'][0]
# gg= torch.rand(956,1002)[[1, 0, 1, 0]] 
# gg.shape

torch.Size([4, 1002])

In [ ]:
# a1 = torch.rand(956,1002)
# a2 = torch.tensor(a1.shape)[[1, 0, 1, 0]] 
# a2.shape

torch.Size([4])

In [ ]:
# torch.Tensor(det_boxes_session1[0]['glove'][0]) / a2

tensor([0.09281, 0.54916, 0.22355, 0.72385])

In [ ]:
# a2 = torch.Tensor([2,4,6,8])
# a1 = torch.Tensor([20,40,60,80,200,400,600,800]) / a2.repeat(2)
# a1
# a2.repeat(2)

tensor([2., 4., 6., 8., 2., 4., 6., 8.])

In [ ]:
# a1 = torch.rand(956,1002)
# gn = torch.tensor(a1.shape)[[1, 0, 1, 0]] 
# bb = torch.zeros(44)
# # det_boxes_session1[0][0]
# start_ind = 0
# if det_boxes_session1[0].get('glove',None) != None:
#     bb[40:44] = torch.Tensor(det_boxes_session1[0]['glove'][0]) / gn
# if det_boxes_session1[0].get('unblemished',None) != None:
#     end_ind = start_ind + len(det_boxes_session1[0]['unblemished'])*4
#     bb[start_ind:end_ind] = torch.Tensor(det_boxes_session1[0]['unblemished']).view(-1) / gn.repeat(len(det_boxes_session1[0]['unblemished']))
#     start_ind = end_ind
# if self.det_boxes[index].get('blemished',None) != None:
#     end_ind = 40
#     start_ind = 40 - len(self.det_boxes[index]['blemished'])*4
#     bb[start_ind:end_ind] = torch.Tensor(self.det_boxes[index]['blemished']).view(1,-1) / gn.repeat(len(self.det_boxes[index]['blemished']))
#     start_ind = end_ind

# bb

tensor([0.07086, 0.77301, 0.08882, 0.08159, 0.58683, 0.76987, 0.07186, 0.08159, 0.83333, 0.84414, 0.06287, 0.07008, 0.53693, 0.81904, 0.07884, 0.08159, 0.40918, 0.80753, 0.07884, 0.08264, 0.22355, 0.73222, 0.07784, 0.07322, 0.23852, 0.78138, 0.07984, 0.07322, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000,
        0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.09281, 0.54916, 0.13074, 0.17469])

In [ ]:
# len(det_boxes_session1[0]['unblemished'])*4

In [ ]:
# det_boxes_session1[0]['glove']

[[93, 525, 131, 167]]

In [ ]:
# torch.Tensor(det_boxes_session1[0]['unblemished']).view(-1)

tensor([ 71., 739.,  89.,  78., 588., 736.,  72.,  78., 835., 807.,  63.,  67., 538., 783.,  79.,  78., 410., 772.,  79.,  79., 224., 700.,  78.,  70., 239., 747.,  80.,  70.])

In [ ]:
# torch.Tensor(det_boxes_session1[0]['unblemished']).view(1,-1)

tensor([[ 71., 739.,  89.,  78., 588., 736.,  72.,  78., 835., 807.,  63.,  67., 538., 783.,  79.,  78., 410., 772.,  79.,  79., 224., 700.,  78.,  70., 239., 747.,  80.,  70.]])

In [ ]:
# !ls ../../

 annos	 jpg_files  'session1_files (1)'  'session2_files (1)'	 yolov5


In [ ]:
# image = Image.open('../../session2_files (1)/session2 (336).jpg')
# image.size[0]

1002

In [5]:
import pandas as pd
df_csv = pd.read_csv('../../annos/data_session1_annos.csv')
data = pd.DataFrame()
data['Images'] = 'session1 ('+df_csv['Image'].astype(str)+').jpg'
data['EE_labels'] = df_csv['EE_loc']
data['O_labels'] = df_csv['O_loc']
data['P_labels'] = df_csv['Prediction']
data.head()

,Images,EE_labels,O_labels,P_labels
0,session1 (1).jpg,3,0,1
1,session1 (2).jpg,3,0,1
2,session1 (3).jpg,0,0,1
3,session1 (4).jpg,0,0,1
4,session1 (5).jpg,0,0,1


In [6]:
batch_size = 32
validation_split = .3
shuffle_dataset = False
random_seed= 42

In [7]:
# Creating data indices for training and validation splits:
# from sklearn.model_selection import train_test_split
# tr, val = train_test_split(data.label, stratify=data.label, test_size=0.1)
dataset_size = len(data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
#train_indices is equivalent to list(tr.index)
#val_indices is equivalent to list(val.index)

In [8]:
# Creating PT data samplers and loaders:
train_sampler = SequentialSampler(train_indices)
valid_sampler = SequentialSampler(val_indices)

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [10]:
class Arthopod_Dataset(Dataset):
    def __init__(self, img_data,img_path,transform=None, det_boxes = []):
        self.img_path = img_path
        self.transform = transform
        self.img_data = img_data
        self.det_boxes = det_boxes
        
    def __len__(self):
        return len(self.img_data)
    
    def __getitem__(self, index):
        img_name = os.path.join(self.img_path,#self.img_data.loc[index, 'labels'],
                                self.img_data.loc[index, 'Images'])
        image = Image.open(img_name)
        #image = image.convert('RGB')
        rows, cols = image.size[1],image.size[0]
        image = image.resize((640,480))
        label = torch.tensor(self.img_data.loc[index,['EE_labels','O_labels','P_labels']])
        # self.transform = True
        if self.transform is not None: #self.transform should be true
            image = self.transform(image)
        gn = torch.tensor([rows, cols])[[1, 0, 1, 0]] 
        bb = torch.zeros(44)
        # det_boxes_session1[0][0]
        start_ind = 0
        if self.det_boxes[index].get('glove',None) != None:
            bb[40:44] = torch.Tensor(self.det_boxes[index]['glove'][0]) / gn
        if self.det_boxes[index].get('unblemished',None) != None:
            end_ind = start_ind + len(self.det_boxes[index]['unblemished'])*4
            bb[start_ind:end_ind] = torch.Tensor(self.det_boxes[index]['unblemished']).view(-1) / gn.repeat(len(self.det_boxes[index]['unblemished']))
            start_ind = end_ind
        if self.det_boxes[index].get('blemished',None) != None:
            end_ind = 40
            start_ind = 40 - len(self.det_boxes[index]['blemished'])*4
            bb[start_ind:end_ind] = torch.Tensor(self.det_boxes[index]['blemished']).view(-1) / gn.repeat(len(self.det_boxes[index]['blemished']))
            start_ind = end_ind
        # print("image.shape:::::",image.shape)
        return image, bb, label


In [ ]:
# !ls '../../session1_files'

In [11]:
img_path = os.path.join('../../session1_files')
dataset = Arthopod_Dataset(data,img_path,transform, det_boxes_session1)

In [13]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

In [14]:
def img_display(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    return npimg

## Neural Net

In [15]:
class Net(nn.Module):
    # def __init__(self):
    #     super(Net, self).__init__()
    #     # 3 input image channel, 16 output channels, 3x3 square convolution kernel
    #     self.conv1 = nn.Conv2d(3,16,kernel_size=3,stride=2,padding=1)
    #     self.conv2 = nn.Conv2d(16, 32,kernel_size=3,stride=2, padding=1)
    #     self.conv3 = nn.Conv2d(32, 64,kernel_size=3,stride=2, padding=1)
    #     self.conv4 = nn.Conv2d(64, 64,kernel_size=3,stride=2, padding=1)
    #     self.pool = nn.MaxPool2d(2, 2)
    #     self.dropout = nn.Dropout2d(0.4)
    #     self.batchnorm1 = nn.BatchNorm2d(16)
    #     self.batchnorm2 = nn.BatchNorm2d(32)
    #     self.batchnorm3 = nn.BatchNorm2d(64)
    #     self.fc1 = nn.Linear(64*8*10,512 )
    #     self.fc2 = nn.Linear(512, 256)
    #     self.fc3 = nn.Linear(256, 4)

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,9), stride=1)
        self.pool1 = nn.MaxPool2d(4, 3)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,9), stride=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=(3,9), stride=1)
        self.pool2 = nn.MaxPool2d(2, 3)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=(3,9), stride=1)
        self.conv5 = nn.Conv2d(32, 32, kernel_size=(3,9), stride=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.fc1_1 = nn.Linear((32*24*24)+44, 128) ### for input shape: [1, 3, 480, 640], the output shape is: [1, 32, 24, 24] and then we concat 44 bounding box
        self.fc1_2 = nn.Linear(128, 64) # Change input dimension
        self.fc1_3 = nn.Linear(64, 32) # Change input dimension
        self.fc1_4 = nn.Linear(32, 4) # Change input  # for ee_loc
        self.fc1_5 = nn.Linear(32, 4) # Change input  # for o_loc
        # self.fc1_6 = nn.Linear(32, 2) # Change input  # for p_label

        
    def forward(self, x, x_bb):
    #     x = self.batchnorm1(F.relu(self.conv1(x)))
    #     x = self.batchnorm2(F.relu(self.conv2(x)))
    #     x = self.dropout(self.batchnorm2(self.pool(x)))
    #     x = self.batchnorm3(self.pool(F.relu(self.conv3(x))))
    #     x = self.dropout(self.conv4(x))
    #     x = x.view(-1, 64*8*10) # Flatten layer
    #     x = self.dropout(self.fc1(x))
    #     x = self.dropout(self.fc2(x))
    #     x = F.log_softmax(self.fc3(x),dim = 1)
    #     return x
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool2(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.pool3(x) ### for input shape: [1, 3, 480, 640], the output shape is: [1, 32, 24, 24]
        
        # print("Shape of x::::", x.shape)
        x = x.view(-1, 32*24*24) # Flatten layer
        # print("x[0,:]::::::::",x[0,:])
        # print("Shape of x::::", x.shape, "Shape of x_bb::::", x_bb.shape)
        x = torch.cat((x, x_bb),1)
        # x = x.view()
        x = self.fc1_1(x)
        x = self.fc1_2(x)
        x = self.fc1_3(x)
        x1 = self.fc1_4(x) ## for ee_loc
        x2 = self.fc1_5(x) ## for o_loc
        # x3 = self.fc1_6(x) ## for p_label
        x1 = F.softmax(x1,dim = 1)
        x2 = F.softmax(x2,dim = 1)
        # x3 = F.softmax(x3,dim = 1)
        return x1,x2 #, x3

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

# model = Net() # On CPU
model = Net().to(device)  # On GPU
print(model)

cuda:0
Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 9), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=4, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 9), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 9), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 32, kernel_size=(3, 9), stride=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(3, 9), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1_1): Linear(in_features=18476, out_features=128, bias=True)
  (fc1_2): Linear(in_features=128, out_features=64, bias=True)
  (fc1_3): Linear(in_features=64, out_features=32, bias=True)
  (fc1_4): Linear(in_features=32, out_features=4, bias=True)
  (fc1_5): Linear(in_features=32, out_features=4, bias=True)
)


In [17]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [18]:
def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

In [19]:
n_epochs = 40
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc1 = []
val_acc2 = []
val_acc3 = []
train_loss = []
train_acc1 = []
train_acc2 = []
train_acc3 = []
total_step = len(train_loader)
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    # scheduler.step(epoch)
    correct1 = 0
    correct2 = 0
    # correct3 = 0
    total1=0
    total2=0
    # total3=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, bb_, target_) in enumerate(train_loader):
        data_, bb_, target_ = data_.to(device), bb_.to(device), target_.to(device)# on GPU
        # print("target_.shape:::",target_.shape)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs1, outputs2 = model(data_, bb_)
        loss1 = criterion(outputs1, target_[:,0])
        loss2 = criterion(outputs2, target_[:,1])
        # loss3 = criterion(outputs3, target_[:,2])
        loss = loss1 + loss2 #+ loss3
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        _,pred1 = torch.max(outputs1, dim=1)
        _,pred2 = torch.max(outputs2, dim=1)
        # _,pred3 = torch.max(outputs3, dim=1)
        correct1 += torch.sum(pred1==target_[:,0]).item()
        correct2 += torch.sum(pred2==target_[:,1]).item()
        # correct3 += torch.sum(pred3==target_[:,2]).item()
        total1 += target_[:,0].size(0)
        total2 += target_[:,1].size(0)
        # total3 += target_[:,2].size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc1.append(100 * correct1 / total1)
    train_acc2.append(100 * correct2 / total2)
    # train_acc3.append(100 * correct3 / total3)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc1: {(100 * correct1 / total1):.4f}, train acc2: {(100 * correct2 / total2):.4f}')
    batch_loss = 0
    total_t1=0
    total_t2=0
    # total_t3=0
    correct_t1=0
    correct_t2=0
    # correct_t3=0
    with torch.no_grad():
        model.eval()
        for data_t, bb_t, target_t in (validation_loader):
            data_t, bb_t, target_t = data_t.to(device), bb_t.to(device), target_t.to(device)# on GPU
            outputs_t1, outputs_t2 = model(data_t, bb_t)
            loss_t1 = criterion(outputs_t1, target_t[:,0])
            loss_t2 = criterion(outputs_t2, target_t[:,1])
            # loss_t3 = criterion(outputs_t3, target_t[:,2])
            loss_t = loss_t1 + loss_t2 #+ loss_t3
            batch_loss += loss_t.item()
            _,pred_t1 = torch.max(outputs_t1, dim=1)
            _,pred_t2 = torch.max(outputs_t2, dim=1)
            # _,pred_t3 = torch.max(outputs_t3, dim=1)
            correct_t1 += torch.sum(pred_t1==target_t[:,0]).item()
            correct_t2 += torch.sum(pred_t2==target_t[:,1]).item()
            # correct_t3 += torch.sum(pred_t3==target_t[:,2]).item()
            total_t1 += target_t[:,0].size(0)
            total_t2 += target_t[:,1].size(0)
            # total_t3 += target_t[:,2].size(0)
        # print("Eval::::correct_t1,total_t1", correct_t1, total_t1 )
        val_acc1.append(100 * correct_t1 / total_t1)
        val_acc2.append(100 * correct_t2 / total_t2)
        # val_acc3.append(100 * correct_t3 / total_t3)
        val_loss.append(batch_loss/len(validation_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc1: {(100 * correct_t1 / total_t1):.4f}, validation acc2: {(100 * correct_t2 / total_t2):.4f}\n')
        # Saving the best weight 
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(model.state_dict(), 'model_classification_tutorial.pt')
            print('Detected network improvement, saving current model')
    model.train()

Epoch 1

Epoch [1/40], Step [0/8], Loss: 2.7786

train loss: 2.7774, train acc1: 40.7258, train acc2: 2.0161
validation loss: 2.7706, validation acc1: 40.9524, validation acc2: 17.1429

Detected network improvement, saving current model
Epoch 2

Epoch [2/40], Step [0/8], Loss: 2.7773

train loss: 2.7764, train acc1: 40.7258, train acc2: 39.1129
validation loss: 2.7691, validation acc1: 40.9524, validation acc2: 40.9524

Detected network improvement, saving current model
Epoch 3

Epoch [3/40], Step [0/8], Loss: 2.7748

train loss: 2.7751, train acc1: 40.7258, train acc2: 40.7258
validation loss: 2.7674, validation acc1: 40.9524, validation acc2: 40.9524

Detected network improvement, saving current model
Epoch 4

Epoch [4/40], Step [0/8], Loss: 2.7718

train loss: 2.7737, train acc1: 40.7258, train acc2: 40.7258
validation loss: 2.7655, validation acc1: 40.9524, validation acc2: 40.9524

Detected network improvement, saving current model
Epoch 5

Epoch [5/40], Step [0/8], Loss: 2.7684



In [ ]:
df_csv_test = pd.read_csv('../../annos/data_session2_annos.csv')
data_test = pd.DataFrame()
data_test['Images'] = 'session2 ('+df_csv_test['Image'].astype(str)+').jpg'
data_test['EE_labels'] = df_csv_test['EE_loc']
data_test['O_labels'] = df_csv_test['O_loc']
data_test['P_labels'] = df_csv_test['Prediction']
data_test.head()

,Images,EE_labels,O_labels,P_labels
0,session2 (1).jpg,3,0,1
1,session2 (2).jpg,3,0,1
2,session2 (3).jpg,0,0,1
3,session2 (4).jpg,0,0,1
4,session2 (5).jpg,0,0,1


In [ ]:
from torch.utils.data.sampler import SequentialSampler
test_dataset_size = len(data_test)
test_indices = list(range(test_dataset_size))
test_sampler = SequentialSampler(test_indices)

In [ ]:
img_path = '../../session2_files'
test_dataset = Arthopod_Dataset(data_test,img_path,transform, det_boxes_session2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, sampler=test_sampler)

In [ ]:
# val_loss = []
test_acc1 = []
test_acc2 = []
# test_acc3 = []

total_t1=0
total_t2=0
# total_t3=0
correct_t1=0
correct_t2=0
# correct_t3=0

allpreds1 = torch.rand(0)
allpreds2 = torch.rand(0)
# allpreds3 = torch.rand(0)

allpreds1 = allpreds1.to(device)
allpreds2 = allpreds2.to(device)
# allpreds3 = allpreds3.to(device)

with torch.no_grad():
    model.eval()
    for data_t, bb_t, target_t in (test_loader):
        data_t, bb_t, target_t = data_t.to(device), bb_t.to(device), target_t.to(device)# on GPU
        outputs_t1, outputs_t2 = model(data_t, bb_t)
        # loss_t1 = criterion(outputs_t1, target_t[:,0])
        # loss_t2 = criterion(outputs_t2, target_t[:,1])
        # loss_t3 = criterion(outputs_t3, target_t[:,2])
        # loss_t = loss_t1 + loss_t2 + loss_t3
        # batch_loss += loss_t.item()
        _,pred_t1 = torch.max(outputs_t1, dim=1)
        # print(pred_t1.shape)
        _,pred_t2 = torch.max(outputs_t2, dim=1)
        # _,pred_t3 = torch.max(outputs_t3, dim=1)

        allpreds1 = torch.cat((allpreds1,pred_t1),dim=0)
        allpreds2 = torch.cat((allpreds2,pred_t2),dim=0)
        # allpreds3 = torch.cat((allpreds3,pred_t3),dim=0)

        correct_t1 += torch.sum(pred_t1==target_t[:,0]).item()
        correct_t2 += torch.sum(pred_t2==target_t[:,1]).item()
        # correct_t3 += torch.sum(pred_t3==target_t[:,2]).item()
        total_t1 += target_t[:,0].size(0)
        total_t2 += target_t[:,1].size(0)
        # total_t3 += target_t[:,2].size(0)
    # print("Eval::::correct_t1,total_t1", correct_t1, total_t1 )
    test_acc1.append(100 * correct_t1 / total_t1)
    test_acc2.append(100 * correct_t2 / total_t2)
    # test_acc3.append(100 * correct_t3 / total_t3)
    # val_loss.append(batch_loss/len(validation_loader))
    # network_learned = batch_loss < valid_loss_min
    print(f'test acc1: {(100 * correct_t1 / total_t1):.4f}, test acc2: {(100 * correct_t2 / total_t2):.4f}\n')


test acc1: 46.6321, test acc2: 63.9896



In [ ]:
n_epochs = 40
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc1 = []
val_acc2 = []
val_acc3 = []
train_loss = []
train_acc1 = []
train_acc2 = []
train_acc3 = []
total_step = len(train_loader)
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    # scheduler.step(epoch)
    correct1 = 0
    correct2 = 0
    correct3 = 0
    total1=0
    total2=0
    total3=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, bb_, target_) in enumerate(train_loader):
        data_, bb_, target_ = data_.to(device), bb_.to(device), target_.to(device)# on GPU
        # print("target_.shape:::",target_.shape)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs1, outputs2, outputs3 = model(data_, bb_)
        loss1 = criterion(outputs1, target_[:,0])
        loss2 = criterion(outputs2, target_[:,1])
        loss3 = criterion(outputs3, target_[:,2])
        loss = loss1 + loss2 + loss3
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        _,pred1 = torch.max(outputs1, dim=1)
        _,pred2 = torch.max(outputs2, dim=1)
        _,pred3 = torch.max(outputs3, dim=1)
        correct1 += torch.sum(pred1==target_[:,0]).item()
        correct2 += torch.sum(pred2==target_[:,1]).item()
        correct3 += torch.sum(pred3==target_[:,2]).item()
        total1 += target_[:,0].size(0)
        total2 += target_[:,1].size(0)
        total3 += target_[:,2].size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc1.append(100 * correct1 / total1)
    train_acc2.append(100 * correct2 / total2)
    train_acc3.append(100 * correct3 / total3)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc1: {(100 * correct1 / total1):.4f}, train acc2: {(100 * correct2 / total2):.4f}, train acc3: {(100 * correct3 / total3):.4f}')
    batch_loss = 0
    total_t1=0
    total_t2=0
    total_t3=0
    correct_t1=0
    correct_t2=0
    correct_t3=0
    with torch.no_grad():
        model.eval()
        for data_t, bb_t, target_t in (validation_loader):
            data_t, bb_t, target_t = data_t.to(device), bb_t.to(device), target_t.to(device)# on GPU
            outputs_t1, outputs_t2, outputs_t3 = model(data_t, bb_t)
            loss_t1 = criterion(outputs_t1, target_t[:,0])
            loss_t2 = criterion(outputs_t2, target_t[:,1])
            loss_t3 = criterion(outputs_t3, target_t[:,2])
            loss_t = loss_t1 + loss_t2 + loss_t3
            batch_loss += loss_t.item()
            _,pred_t1 = torch.max(outputs_t1, dim=1)
            _,pred_t2 = torch.max(outputs_t2, dim=1)
            _,pred_t3 = torch.max(outputs_t3, dim=1)
            correct_t1 += torch.sum(pred_t1==target_t[:,0]).item()
            correct_t2 += torch.sum(pred_t2==target_t[:,1]).item()
            correct_t3 += torch.sum(pred_t3==target_t[:,2]).item()
            total_t1 += target_t[:,0].size(0)
            total_t2 += target_t[:,1].size(0)
            total_t3 += target_t[:,2].size(0)
        # print("Eval::::correct_t1,total_t1", correct_t1, total_t1 )
        val_acc1.append(100 * correct_t1 / total_t1)
        val_acc2.append(100 * correct_t2 / total_t2)
        val_acc3.append(100 * correct_t3 / total_t3)
        val_loss.append(batch_loss/len(validation_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc1: {(100 * correct_t1 / total_t1):.4f}, validation acc2: {(100 * correct_t2 / total_t2):.4f}, validation acc3: {(100 * correct_t3 / total_t3):.4f}\n')
        # Saving the best weight 
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(model.state_dict(), 'model_classification_tutorial.pt')
            print('Detected network improvement, saving current model')
    model.train()

Epoch 1



ValueError: ignored